In [6]:
"""
不变性: 在图片中的哪一个位置都是可以的
卷积核:周围像素点对当前这个像素点的影响
卷积层将输入和核矩阵进行交叉相关，加上偏移后得到输出 核矩阵和偏移都是可以学习的参数 
以下是自定义的卷积层
"""
import torch
from torch import  nn
# from d2l import torch as d2l

def corr2d(X, K):   # X 为输入，K为核矩阵
    """计算二维互相关信息 这个是计算上的定义"""
    h, w = K.shape # 核矩阵的行和列
    print(X, end="\n\n")
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    print(Y)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i + h, j:j + w] * K).sum() # 图片的小方块区域与卷积核做点积
           # Y[i, j] = (X[i:i + h, j:j + w] * K).sum()
    return Y

# 验证上述二维互相关运算的输出
X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K = torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

tensor([[0., 0.],
        [0., 0.]])


tensor([[19., 25.],
        [37., 43.]])

In [11]:
# 实现自定义的二维卷积层
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
        
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias
    
# 卷积层的一个简单应用：检测图片中不同颜色的边缘
X = torch.ones((6,8))
X[:,2:6] = 0  # 把中间四列设置为0  0和1表示为过度 表示边缘

K = torch.tensor([[1.0,-1.0]])  # 如果左右原值相等，那么这两原值乘1和-1相加为0，则不是边缘
Y = corr2d(X, K)
Y

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

tensor([[0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0.]])


tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [12]:
X = torch.ones((6,8))
X[:,2:6] = 0 # 将第2-5列全部设置为0
print(corr2d(X.t(), K)) # X.t() 为X的转置，而K卷积核只能检测垂直边缘

tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])


In [15]:
# 学习由X生成Y的卷积核
conv2d = nn.Conv2d(1, 1, kernel_size=(1,2), bias=False) # 单个矩阵，输入通道为1，黑白图片通道为1，彩色图片通道为3。这里输入通道为1，输出通道为1.   
X = X.reshape((1,1,6,8)) # 通道维：通道数，RGB图3通道，灰度图1通道，批量维就是样本维，就是样本数
Y = Y.reshape((1,1,6,7))
for i in range(10):
    Y_hat = conv2d(X)
    loss = (Y_hat - Y) ** 2 
    conv2d.zero_grad() # 所有参数的梯度初始化为0 
    loss.sum().backward() # 梯度只能为标量（即一个数）输出隐式地创建 这个是必须要的
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad # 3e-2是学习率
    if(i+1) % 2 == 0:
        print(f'batch {i+1},loss {loss.sum():.3f}')
        
# 所学的卷积核的权重张量
print(conv2d.weight.data.reshape((1,2)))


batch 2,loss 15.872
batch 4,loss 5.410
batch 6,loss 2.033
batch 8,loss 0.802
batch 10,loss 0.323
tensor([[ 0.9306, -1.0474]])
